# Init Environment

## Note

In [ ]:
# note of the module installations
# install python with pyenv (ref: https://github.com/lewagon/data-setup/blob/e5a239926f304d452704718136b6e2f017c7303d/macOS.md#installing-python-with-pyenv)
# setup virtual environment with pyenv (ref:https://github.com/lewagon/data-setup/blob/e5a239926f304d452704718136b6e2f017c7303d/macOS.md#installing-python-with-pyenv
# error: 'Failed to activate virtualenv' (ref:https://github.com/pyenv/pyenv-virtualenv/issues/387)
# pip install fasttext (0.9.2) (after installing the dependencies: numpy, scipy, pybind11, setuptools, wheel)

# install Jupyter notebook and nbextensions (ref: https://github.com/lewagon/data-setup/blob/e5a239926f304d452704718136b6e2f017c7303d/macOS.md#jupyter-notebook-extensions)
#   pip install jupyter_contrib_nbextensions
#   pip install --upgrade notebook==6.4.12 (ref:https://stackoverflow.com/questions/49647705/jupyter-nbextensions-does-not-appear)
#   pip install traitlets==5.9.0

## Import modules

In [5]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from tqdm import tqdm
import concurrent.futures

import sys
sys.path.append('../src/')

from config import *

from src.helper_visualization import *
from src.helper_text import *
from src.helper_language import *
from src.helper_translation import *
from src.helper_pred import *

# Combine Data Source

In [ ]:
import glob

# Get a list of file paths that match the pattern
files = glob.glob(f'{DATA_FOLDER_PATH_RAW}/data_202*.xlsx')
# Create an empty list to store DataFrames
dfs = []

# Loop through each file and read it into a DataFrame
for file in files:
    df = pd.read_excel(file, index_col=None)
    dfs.append(df)

# Concatenate the DataFrames into one
df_combined = pd.concat(dfs, ignore_index=True)
df_combined.info()

In [ ]:
#check for null values
df_combined.isnull().sum().sort_values(ascending = False)

In [ ]:
# columns which have large number of null values will be dropped 
columns_to_drop = [
    'Escalated To Engineering', 
    'Bug Type', 
    'Status Reason', 
    'Escalated to L2',
    'Category',
    'Completion Code'
]
try: 
    df_combined.drop(columns_to_drop, axis=1, inplace = True)
    df_combined.dropna(subset=['Request ID'], inplace=True)
    df_combined.dropna(subset=['Product Name'], inplace=True)
    df_combined.dropna(subset=['Title'], inplace=True)
except:
    pass
# surprisingly, there are over 100k duplications
df_combined.drop_duplicates(subset=['Title', 'Product Name'], keep='first', inplace=True)

In [ ]:
df_combined.isnull().sum().sort_values(ascending = False)

In [ ]:
# Specify the file name and path
excel_file = f'{DATA_FOLDER_PATH_PROCESSED}/data_combined.xlsx'

# Export the DataFrame to Excel
df_combined.to_excel(excel_file, index=False)  # Set index to False if you don't want to export the DataFrame index

# Load Data for Pre-processing

In [ ]:
data_url = f'{DATA_FOLDER_PATH_PROCESSED}/data_combined.xlsx'
df_combined = pd.read_excel(data_url, index_col=None)
print(df_combined.info())

In [ ]:
df_processed = df_combined.copy()
df_processed['Length'] = 0
df_processed['Language']=pd.NA
df_processed['Title_Processed']=pd.NA
df_processed['Title_Translated']=pd.NA
df_processed['Title_Cleaned']=pd.NA
print(df_processed.info())
df_processed.head()

# Cleanup Product Name

In [ ]:
hist_by_labels(df_processed, 'Product Name', log=True, horizontal=True)

In [ ]:
# Use Module Name to replace Product Name for all 'Petrotechnical Suite - Domain Profiles'
df_processed.loc[df_processed['Product Name'] == 'Petrotechnical Suite - Domain Profiles', 'Product Name'] = df_processed['Module Name']
df_processed.drop(columns=['Module Name'], inplace=True)
df_processed.dropna(subset=['Product Name'], inplace=True)

# Consolidate the following products into Petrel
df_processed.loc[df_processed['Product Name'] == 'Petrel Exploration Geology', 'Product Name'] = 'Petrel'
df_processed.loc[df_processed['Product Name'] == 'Petrel Project Explorer', 'Product Name'] = 'Petrel'

# Case of one product has multiple names
df_processed.loc[df_processed['Product Name'] == 'DELFI RE', 'Product Name'] = 'Petrel RE' # Pretel RE --> DELFI RE
df_processed.loc[df_processed['Product Name'] == 'OMEGA', 'Product Name'] = 'Omega' # OMEGA --> Omega
df_processed.loc[df_processed['Product Name'] == 'Other', 'Product Name'] = 'Others' # Other --> Others
df_processed.loc[df_processed['Product Name'] == 'Third-Party Applications', 'Product Name'] = 'App – Third Party' # Third-Party Applications --> App – Third Party
df_processed.loc[df_processed['Product Name'] == 'Ocean Plug-ins for Petrel - Third party', 'Product Name'] = 'Ocean Plug-ins for Petrel - Third Party' # Ocean Plug-ins for Petrel - Third party --> Ocean Plug-ins for Petrel - Third Party

# Consolidate the following products into 'Deployment'
df_processed.loc[df_processed['Product Name'] == 'Provisioning & Decommissioning', 'Product Name'] = 'Deployment'
df_processed.loc[df_processed['Product Name'] == 'Software Demo and Evaluation', 'Product Name'] = 'Deployment'
df_processed.loc[df_processed['Product Name'] == 'Internal Deployment', 'Product Name'] = 'Deployment'
df_processed.loc[df_processed['Product Name'] == 'Image', 'Product Name'] = 'Deployment'

# Consolidate the following products into 'Delfi Portal'
df_processed.loc[df_processed['Product Name'] == 'SAuth', 'Product Name'] = 'Delfi Portal'
df_processed.loc[df_processed['Product Name'] == 'License', 'Product Name'] = 'Delfi Portal'
df_processed.loc[df_processed['Product Name'] == 'Licensing', 'Product Name'] = 'Delfi Portal'
df_processed.loc[df_processed['Product Name'] == 'Environment', 'Product Name'] = 'Delfi Portal'
df_processed.loc[df_processed['Product Name'] == 'Authorization', 'Product Name'] = 'Delfi Portal'

#drop the rows which Product Name is 'Quality and Feedback' and 'Software Training Services'
df_processed = df_processed[df_processed['Product Name'] != 'Quality and Feedback']
df_processed = df_processed[df_processed['Product Name'] != 'Software Training Services']

#Drop the products which have less than 50 records
df_processed = df_processed.groupby('Product Name').filter(lambda x : len(x)>50)

print(df_processed.info())
hist_by_labels(df_processed, 'Product Name', top=None, log=True, horizontal=True)

# Clean-up Title Column

## Quick Clean-up

In [ ]:
# Set the number of rows to process
num = df_processed['Title'].notnull().sum()

# Initialize a progress bar with the total number of rows
progress_bar = tqdm(total=num, desc="Processing Rows", unit=" row")

# Function to process a single row and update the 'Processed_Title' column
def process_row(index):
    # processed_title = preprocess_step_1(df_combined.at[index, 'Title'])
    processed_title = quick_clean_up(df_processed.at[index, 'Title'])
    df_processed.at[index, 'Title_Processed'] = processed_title
    df_processed.at[index, 'Length'] = len(processed_title)
#     df.at[index, 'Language'] = detect_language_fasttext(processed_title)
    progress_bar.update(1)

# Define the number of parallel workers (adjust this based on your CPU cores)
num_workers = 8

# Create a ThreadPoolExecutor with the specified number of workers
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use the executor to process rows in parallel
    executor.map(process_row, df_processed.index)

# Close the progress bar
progress_bar.close()

In [ ]:
# Delete records with missing values in 'ProductName' columns
df_processed.dropna(subset=['Title_Processed'], inplace=True)
# Remove duplicates based on 'Title' and 'ProductName' columns
df_processed.drop_duplicates(subset=['Title_Processed', 'Product Name'], keep='first', inplace=True)

print(df_processed.info())
hist_by_labels(df_processed, 'Length', log=False)

In [ ]:
# Set the number of rows to process
num = df_processed['Title_Processed'].notnull().sum()

# df['Title_Translated'] = df['Title_Processed']
# Initialize a progress bar with the total number of rows
progress_bar = tqdm(total=num, desc="Processing Rows", unit=" row")

# Function to process a single row and update the 'Processed_Title' column
def process_row(index):
    df_processed.at[index, 'Language'] = detect_language_fasttext(df_processed.at[index, 'Title_Processed'])
    progress_bar.update(1)

# Define the number of parallel workers (adjust this based on your CPU cores)
num_workers = 8

# Create a ThreadPoolExecutor with the specified number of workers
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use the executor to process rows in parallel
    executor.map(process_row, df_processed.index)

# Close the progress bar
progress_bar.close()


In [ ]:
# fasttext performance is better than the other solutions for short text 
# (ref: https://medium.com/besedo-engineering/language-identification-for-very-short-texts-a-review-c9f2756773ad

hist_by_labels(df_processed, "Language", log=True, horizontal=True, left=10.5)
hist_by_labels(df_processed, "Language", top=20, log=True, horizontal=True)

In [ ]:
# Specify the file name and path
excel_file = f'{DATA_FOLDER_PATH_PROCESSED}/data_processed.xlsx'

# Export the DataFrame to Excel
df_processed.to_excel(excel_file, index=False)  # Set index to False if you don't want to export the DataFrame index

## Language Detection

In [ ]:
excel_file = f'{DATA_FOLDER_PATH_PROCESSED}/data_processed.xlsx'
df_processed = pd.read_excel(excel_file)

df_translated = df_processed.copy()
df_translated['Title_Translated'] = df_translated['Title_Processed']
print(df_translated.info())
df_translated.head(-10)

In [ ]:
# Keep the top 10 languages and drop the rest and the unknown. 
# why? There are high chances the detected language were not correct due to wrong spelling, etc.

language_counts = df_translated['Language'].value_counts()
cutoff = 99
language_others = language_counts.index[language_counts < cutoff]

mask_others = df_translated['Language'].isin(language_others)
df_translated.loc[mask_others,'Language']='unknown'

# Drop rows where 'Language' is equal to 'unknown'
df_translated = df_translated[df_translated['Language'] != 'unknown']
print(df_translated.info())
hist_by_labels(df_translated, 'Language', log=True, horizontal=True)

## Translation of Non-English

In [ ]:
# Group the DataFrame by the "Language" column
grouped = df_translated.groupby("Language")

# Initialize tqdm to display progress bar
pbar = tqdm(total=len(grouped), desc=f"Processing")

# Create an empty list to store the processed groups
processed_groups = []

# Iterate through sub DataFrames
for lang, group in grouped:
    pbar.set_description(f"Processing [{lang}/{len(group)}]")

    if lang != "en": # Skip English
        translated_titles = translate_array_to_english(
            group["Title_Translated"].tolist(),
            src_lang=lang,
            length_limit=1250)
        group["Title_Translated"] = translated_titles  # Update "Title_Translated" column in the group
    
    processed_groups.append(group)
    pbar.update(1)  # Update the progress bar

# Close the progress bar
pbar.close()

# Concatenate the groups back into a single DataFrame
df_translated = pd.concat(processed_groups, ignore_index=True)

In [ ]:
print(df_translated.isnull().sum().sort_values(ascending = False))

# Delete records with missing values in 'ProductName' columns
df_translated.dropna(subset=['Title_Translated'], inplace=True)
# Remove duplicates based on 'Title' and 'ProductName' columns
df_translated.drop_duplicates(subset=['Title_Processed', 'Product Name'], keep='first', inplace=True)
print(df_translated.info())
hist_by_labels(df_translated, 'Language', log=True, horizontal=True)

In [ ]:
# Specify the file name and path
excel_file = f'{DATA_FOLDER_PATH_PROCESSED}/data_translated.xlsx'

# Export the DataFrame to Excel
df_translated.to_excel(excel_file, index=False)  # Set index to False if you don't want to export the DataFrame index

## Final Clean up

In [ ]:
excel_file = f'{DATA_FOLDER_PATH_PROCESSED}/data_translated.xlsx'

df_cleaned = pd.read_excel(excel_file , dtype={'Request ID': str, 'Title_Cleaned': str})
print(df_cleaned.info())
print(df_cleaned.info())
df_cleaned.head(10)

In [ ]:
df_cleaned.drop(columns=[
    'Created Time', 
    'Customer Company', 'Customer Country', 
    'Priority', 'Urgency', 'Impact', 
    'Service Definition', 'Service Desk Group', 'Status',
    'Closed Time', 
    'Response Time (Min)', 'Resolution Time (Min)', 
    'Contracts Reference', 'Creation Source'
    ], inplace=True)

# Set the number of rows to process
num = df_cleaned['Title_Translated'].notnull().sum()

# Initialize a progress bar with the total number of rows
progress_bar = tqdm(total=num, desc="Processing Rows", unit=" row")

# Function to process a single row and update the 'Processed_Title' column
def process_row(index):
    df_cleaned.at[index, 'Title_Cleaned'] = final_clean_up(df_cleaned.at[index, 'Title_Translated'])
    df_cleaned.at[index, 'Length'] = count_words(df_cleaned.at[index, 'Title_Cleaned'])
    progress_bar.update(1)

# Define the number of parallel workers (adjust this based on your CPU cores)
num_workers = 8

# Create a ThreadPoolExecutor with the specified number of workers
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    # Use the executor to process rows in parallel
    executor.map(process_row, df_cleaned.index)

# Close the progress bar
progress_bar.close()

print(df_cleaned.info())

In [ ]:
# Remove duplicates based on 'Title' and 'ProductName' columns after translation
df_cleaned.dropna(subset=['Title_Cleaned'], inplace=True)
df_cleaned.drop_duplicates(subset=['Title_Processed', 'Product Name'], keep='first', inplace=True)
print(df_cleaned.info())

# remove the same title pointing to multiple products
n_title = df_cleaned['Title_Cleaned'].value_counts()
good_title = n_title.index[n_title == 1]
print (len(good_title))

mask = df_cleaned['Title_Cleaned'].isin(good_title)
df_cleaned = df_cleaned[mask]
print(df_cleaned.info())
print(df_cleaned.isnull().sum().sort_values(ascending = False))

In [ ]:
# Specify the file name and path
excel_file = f'{DATA_FOLDER_PATH_PROCESSED}/data_cleaned.xlsx'

# Export the DataFrame to Excel
df_cleaned.to_excel(excel_file, index=False)  # Set index to False if you don't want to export the DataFrame index

# Training

## Prepare Data for Modeling

### Load Cleaned Data

In [ ]:
excel_file = f'{DATA_FOLDER_PATH_PROCESSED}/data_cleaned.xlsx'
df_cleaned = pd.read_excel(excel_file)
print(df_cleaned.info())
hist_by_labels(df_cleaned, 'Length', log=False, left=2.5, right=15.5)
# plot a square in red color on the histogram

### Remove short Titles

In [ ]:
# filter out the titles with length less than 3 and more than 20
# when the title is too short, it is either not useful or it is too obvious to be classified
mask = (df_cleaned['Length'] > 3) & (df_cleaned['Length'] < 20)
df_cleaned = df_cleaned[mask]
hist_by_labels(df_cleaned, 'Length', horizontal=True)

### Combine the long tail into Others

In [ ]:
hist_by_labels(df_cleaned, 'Product Name', top=26, log=True, horizontal=False)

In [ ]:
# Keep the top 50 products and combine the rest into 'Others'
product_counts = df_cleaned['Product Name'].value_counts()
cutoff = 400
product_top = product_counts.index[product_counts >= cutoff]
product_others = product_counts.index[product_counts < cutoff]

mask_others = df_cleaned['Product Name'].isin(product_others)
df_cleaned.loc[mask_others,'Product Name']='Others'
hist_by_labels(df_cleaned, 'Product Name', log=True)

### Train/Test Split

In [ ]:
# for the products with more than 5000 records, randomly select 4000 records for each product and put in a new dataframe df_cleaned_balanced, then keep the rest of the records in df_cleaned_test
df_cleaned_balanced = pd.DataFrame()
df_cleaned_test = pd.DataFrame()
for product in df_cleaned['Product Name'].unique():
    df_product = df_cleaned[df_cleaned['Product Name'] == product]
    if df_product.shape[0] > 5000:
        df_product_balanced = df_product.sample(4000).copy()
        df_cleaned_balanced = pd.concat([df_cleaned_balanced, df_product_balanced])
        df_product_test = df_product.drop(df_product_balanced.index)
        df_cleaned_test = pd.concat([df_cleaned_test, df_product_test])
    else:
        df_product_balanced = df_product.sample(frac=0.8).copy()
        df_cleaned_balanced = pd.concat([df_cleaned_balanced, df_product_balanced])
        df_product_test = df_product.drop(df_product_balanced.index)
        df_cleaned_test = pd.concat([df_cleaned_test, df_product_test])

print(df_cleaned_balanced.info())
print(df_cleaned_test.info())

hist_by_labels(df_cleaned_balanced, 'Product Name', log=True)
hist_by_labels(df_cleaned_test, 'Product Name', log=True)

In [ ]:
# Prepare data for training and testing for 
# a. cross validation
# b. logistic regression, 
# c. SGC Classifier

#import packages related to data
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

X = df_cleaned['Title_Cleaned']
vectorizer = CountVectorizer(max_features=20000, analyzer='word', ngram_range=(1, 2))
vectorizer.fit(X)

X_train = df_cleaned_balanced['Title_Cleaned']
X_train_encoded = vectorizer.transform(X_train)
y_train = df_cleaned_balanced['Product Name']

X_test = df_cleaned_test['Title_Cleaned']
X_test_encoded = vectorizer.transform(X_test)
y_test = df_cleaned_test['Product Name']

## Model Baseline - Cross Validation

In [ ]:
# Model Tuning
from sklearn.model_selection import cross_validate

# Models
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
# import xgboost as xgb

import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
models = {
    'MultinomialNB': MultinomialNB(),
    'SGDClassifier': SGDClassifier(max_iter=5000),
    'LogisticRegression': LogisticRegression(max_iter=5000),
    'SVC rbf': SVC(kernel='rbf'),
    'SVC linear': SVC(kernel='linear'),
    # 'RandomForestClassifier': RandomForestClassifier(),
    # 'GradientBoostingClassifier': GradientBoostingClassifier(),
    # 'XGBClassifier': xgb.XGBClassifier()
}

print('Baseline Score(s) of each model are ....')

for model in models:
    cv_result = cross_val_score(
        models[model], 
        X_train_encoded, # please provide the features after preprocessing
        y_train, #the target
        cv=5, 
        scoring='accuracy', # lease revise to the appropriate score
        n_jobs=-1
    )
    print(f'Average score: \033[94m{"{:.4f}".format(np.mean(cv_result))}\033[0m by \033[94m{model}\033[0m.')

# Hpyerparameter Tuning

## SGC Classifier

In [ ]:
# Define an objective function to optimize
def objective(trial):

    # Define hyperparameter search space

    pamams = {
        'loss': trial.suggest_categorical('loss', ['log_loss', 'modified_huber']),
        'alpha': trial.suggest_float('alpha', 1e-6, 1e+2, log=True),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'max_iter': 5000,
        'random_state': 42
    }
    
    # Create and train the SGD Classifier with suggested hyperparameters
    sgd_classifier = SGDClassifier(**pamams)

    # Evaluate the model on the validation set
    scores = cross_val_score(sgd_classifier, X_train_encoded, y_train, cv=5, scoring='accuracy')

    return np.mean(scores)

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(
    objective,
    n_trials=100, 
    n_jobs=-1, 
    show_progress_bar=True
    )  # You can adjust the number of trials

# Print the best hyperparameters and corresponding accuracy
best_params = study.best_params
best_accuracy = study.best_value
# Create and train the best Logistic Regression model
best_model_sgc = SGDClassifier(**best_params)
best_model_sgc.fit(X_train_encoded, y_train)
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

In [ ]:
# Predict categories for the test data
y_pred_sgc = best_model_sgc.predict(X_test_encoded)
# Evaluate the classifier's performance
print(classification_report(y_test, y_pred_sgc, digits=3))
plot_confusion_matrix(y_test, y_pred_sgc, title='Confusion matrix for Rests')
df_pred_top5_sgc = top5_accuracy_report(best_model_sgc, vectorizer, X_test, y_test, pre_processed=True)

In [ ]:
# export the model
import pickle
pickle.dump(best_model_sgc, open(f'{MODEL_FOLDER_PATH}/model_sgc_classifier.pkl', 'wb'))

# export the vectorizer
pickle.dump(vectorizer, open(f'{MODEL_FOLDER_PATH}/vectorizer_sgc_classifier.pkl', 'wb'))


## Logistic Classification

In [ ]:
# weights = df_rests['ProductName'].value_counts().apply(np.sqrt).to_dict()
model_logReg = LogisticRegression(
    max_iter=5000,
    # class_weight= weights
)
model_logReg.fit(X_train_encoded, y_train)

# Step 4: Model Evaluation
y_pred_logReg = model_logReg.predict(X_test_encoded)
accuracy = accuracy_score(y_test, y_pred_logReg)
print(f"Accuracy: {accuracy:.3f}")

report = classification_report(y_test, y_pred_logReg, digits=3)
print(report)

plot_confusion_matrix(y_test, y_pred_logReg, title='Confusion matrix for Rests')
df_pred_top5_logReg = top5_accuracy_report(model_logReg, vectorizer, X_test, y_test, pre_processed=True)

## MultinomialNB

In [ ]:
# Define an objective function to optimize
def objective(trial):

    # Define hyperparameter search space of MultinomialNB model
    pamams = {
        'alpha': trial.suggest_float('alpha', 1e-6, 1e+2, log=True),
        'fit_prior': trial.suggest_categorical('fit_prior', [True, False])
    }
    
    # Create and train the SGD Classifier with suggested hyperparameters
    model_multinomialNB = MultinomialNB(**pamams)

    # Evaluate the model on the validation set
    scores = cross_val_score(model_multinomialNB, X_train_encoded, y_train, cv=5, scoring='accuracy')

    return np.mean(scores)

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(
    objective,
    n_trials=300, 
    n_jobs=-1, 
    show_progress_bar=True
    )  # You can adjust the number of trials

# Print the best hyperparameters and corresponding accuracy
best_params = study.best_params
best_accuracy = study.best_value

# Create and train the best Logistic Regression model
best_model_multinomialNB = MultinomialNB(**best_params)
best_model_multinomialNB.fit(X_train_encoded, y_train)
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

In [ ]:
# weights = df_rests['ProductName'].value_counts().apply(np.sqrt).to_dict()
model_multinomialNB = best_model_multinomialNB

# Step 4: Model Evaluation
y_pred_multinomialNB = model_multinomialNB.predict(X_test_encoded)
accuracy = accuracy_score(y_test, y_pred_multinomialNB)
print(f"Accuracy: {accuracy:.3f}")

report = classification_report(y_test, y_pred_multinomialNB, digits=3)
print(report)

plot_confusion_matrix(y_test, y_pred_multinomialNB, title='Confusion matrix for Rests')
df_pred_top5_multinomialNB = top5_accuracy_report(model_multinomialNB, vectorizer, X_test, y_test, pre_processed=True)